In [1]:
import numpy as np
import glob
import matplotlib.pyplot as plt
import sys
# sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, AveragePooling2D, Activation, Input, Flatten, concatenate, GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import initializers
import numpy as np

In [2]:
from functools import partial
from bayes_opt import BayesianOptimization

In [18]:
class WeightedAveragePooling(Layer):
    def __init__(self, output_shape, **kwargs):
        self.shape = output_shape
        super(WeightedAveragePooling, self).__init__(**kwargs)

    def build(self, input_shape):
        self.w = self.add_weight(name='W1', shape=self.shape, initializer='uniform') # creating W

        super(WeightedAveragePooling, self).build(input_shape)

    def call(self, input_):
        w_absolute = K.abs(self.w)  # making w values positive
        numerator = input_*w_absolute
        numerator_sum = K.expand_dims(K.sum(numerator, axis=(1, 2, 3)))
        denominator = K.sum(w_absolute, axis=(1, 2, 3))
        denominator_sum = K.expand_dims(K.sum(w_absolute, axis=(1, 2, 3)))
        return numerator_sum / (denominator_sum + 1e-7)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 1)
    
    def get_config(self):
        config = {
            'shape': self.shape,
        }
        base_config = super(WeightedAveragePooling, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



In [21]:
class SWAP(Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(SWAP, self).__init__(**kwargs)

    def build(self, input_shape):

        # print(input_shape[-1], self.output_shape_)
        input_dim = input_shape[-1]
        print(type(input_dim))

        self.w = self.add_weight(name='w', shape=(input_dim, self.output_dim), initializer='uniform')
        super(SWAP, self).build(input_shape)

    def call(self, inputs):
        w_ = np.abs(self.w)
        self.w = self.w/(np.sum(w_) + 1e-7)
        x = K.dot(inputs, K.abs(self.w))          # weights need to be non negative
        bias_ = -0.5*np.ones(1,)
        output = x + bias_
        output = Activation('sigmoid')(output)

        return output

    def compute_output_shape(self, input_shape):
        shape = list(input_shape)
        shape[-1] = self.output_dim
        return tuple(shape)
    
    def get_config(self):
        config = {'output_dim': self.output_dim
                 }
        base_config = super(SWAP, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


In [22]:
class EyeCalc:
    def __init__(self, input_):
        self.input = input_

    def convolution(self, input_, kernel_size, filters, strides=1, activation='relu', max_pool="True", batch_norm="True"):

        x = Conv2D(kernel_size=kernel_size, filters=filters, strides=strides, padding='same')(input_)

        if activation == 'sigmoid':
            x = Activation('sigmoid')(x)
        else:
            x = Activation('relu')(x)
            #x = LeakyReLU(0.03)(x)

        if batch_norm:
            x = BatchNormalization()(x)

        if max_pool:
            x = MaxPooling2D((2, 2))(x)

        return x

    def conv2d(self):
        num_filters = 64
        x = self.convolution(self.input, 3, num_filters, strides=1)
        for i in range(3):
            num_filters *= 2
            x = self.convolution(x, 3, num_filters, strides=1)
        ### check###

        # x = self.convolution(x, kernel_size=1, filters=num_filters, strides=1, activation='sigmoid', max_pool=False,
        #                      batch_norm=False)
        x = self.convolution(x, kernel_size=1, filters=1, strides=1, activation='sigmoid', max_pool=False, batch_norm=False)

        return x

    def pooling(self, input_, type='wap'):
        if type == 'swap':
            x = Flatten()(input_)
            x = SWAP(1)(x)

        else:
            x = WeightedAveragePooling((1, 31, 31, 1))(input_)

        return x

    def forward(self):
        x = self.conv2d()
        x = self.pooling(x, type='swap')

        return x

    def build_model(self):

        output = self.forward()
        model = Model(self.input, output)

        return model


In [6]:
dir = '/home/Sudhakar/Desktop/'
imgs_path_good = glob.glob(os.path.join(dir, 'Good/*.jpg'))
# print(np.shape(imgs_path_good))
imgs_path_bad = glob.glob(os.path.join(dir, 'Bad/*.jpg'))
# print(np.shape(imgs_path_bad))
images_good= []
images_bad = []
labels_good = []
labels_bad = []

img_size = 512

In [7]:
def image_preprocessing(image):
    img = image
    img = cv2.resize(img, (img_size, img_size))

    return img


def read_imgs(image_path, good=True):
    image_array = []
    label_array = []
    for image in image_path:
        img = plt.imread(image)
        image_array.append(image_preprocessing(img))
        if good:
            label_array.append(0)
        else:
            label_array.append(1)

    return image_array, label_array



In [8]:

images_good, labels_good = read_imgs(imgs_path_good)
images_bad, labels_bad = read_imgs(imgs_path_bad, good=False)

In [9]:

X = images_good + images_bad
Y = labels_good + labels_bad
# print(np.shape(X), np.shape(Y))
X_final = np.reshape(X, (np.shape(X)[0], np.shape(X)[1], np.shape(X)[2], 3))
Y_final = np.reshape(Y, (np.shape(Y)[0], 1))

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_final, Y_final, test_size=0.3)
x_val, x_test2, y_val, y_test_2 = train_test_split(x_test, y_test, test_size=0.2)

In [23]:
np.shape(x_train)

(135, 512, 512, 3)

In [13]:
datagen = ImageDataGenerator(

        vertical_flip=True,
        horizontal_flip=True,
        rotation_range=20,
        rescale=1./255
        #width_shift_range=0.1,
        #height_shift_range=0.1
)
datagen.fit(x_train)
#datagen.fit(x_test)
#datagen.fit(x_val)

In [24]:
checkpoint = ModelCheckpoint('model_512-{epoch:03d}.h5',
                                monitor='val_accuracy', verbose=1, save_best_only=True)
callbacks_list = [checkpoint]

In [25]:
obj = EyeCalc(Input(shape=(img_size, img_size, 3)))
model = obj.build_model()
model.summary()
optimizer = Adam(lr=0.001)

model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

<class 'int'>
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 512, 512, 64)      1792      
_________________________________________________________________
activation_5 (Activation)    (None, 512, 512, 64)      0         
_________________________________________________________________
batch_normalization_v2_4 (Ba (None, 512, 512, 64)      256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 256, 256, 128)     73856     
_________________________________________________________________
activation_6 (Activation)    (None, 256, 256,

In [26]:
batch_size = 16
epochs = 10

In [29]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs, validation_data=(x_val, y_val), callbacks=callbacks_list, steps_per_epoch=200)

Epoch 1/10
199/200 [============================>.] - ETA: 0s - loss: 9.8460 - accuracy: 0.4534
Epoch 00001: val_accuracy improved from -inf to 0.72340, saving model to model_512-001.h5
200/200 [==============================] - 132s 662ms/step - loss: 9.7993 - accuracy: 0.4550 - val_loss: 0.6031 - val_accuracy: 0.7234
Epoch 2/10
199/200 [============================>.] - ETA: 0s - loss: 4.6337 - accuracy: 0.5754
Epoch 00002: val_accuracy did not improve from 0.72340
200/200 [==============================] - 141s 705ms/step - loss: 4.6107 - accuracy: 0.5760 - val_loss: 45.9736 - val_accuracy: 0.2766
Epoch 3/10
199/200 [============================>.] - ETA: 0s - loss: 2.6388 - accuracy: 0.5985
Epoch 00003: val_accuracy did not improve from 0.72340
200/200 [==============================] - 133s 663ms/step - loss: 2.6287 - accuracy: 0.5983 - val_loss: 0.6518 - val_accuracy: 0.6596
Epoch 4/10
199/200 [============================>.] - ETA: 0s - loss: 1.1023 - accuracy: 0.6564
Epoch 0000

In [31]:
y_evaluted = model.evaluate(x_test2, y_test_2)

12/12 [==============================] - 1s 78ms/sample - loss: 0.5991 - accuracy: 0.7500
